### IMPORT

In [95]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime

### 채권구분
- 아래에서 전체 원데이터 만들기
- 새 채권구분 만들기 : 1) 종결, 2) 조정(개회,신복이 중복되더라도 일단 문제없게), 3) 무담보 순서대로 만들기
  - (오류 체크) 개회, 신복 기준데이터 vs 차주 채무자상태 
    - (전산 미수정) 조정 vs 미반영, 면책 및 완납 vs 미반영
    - (개회신복경쟁) 개회 vs 신복 or 신복 vs 개회
    - (오등록) 확정리스트 없음 vs 개회, 신복. 다음의 경우를 포함함[폐지 및 실효 vs 미반영(여전히 개회 및 신복 *개회폐지 후 신복, 신복폐지 후 개회일 수는 있음)]

- (오류 체크) 개회, 신복 보증인상태 미반영 리스트 만들기, 중복리스트 만들키
- 개회, 신복 조정여부 칼럼 추가해서 자산정리 칼럼 만들기(계좌키에 붙이기) : 생략하지 말고 다 붙이기
- 채권구분칼럼 최초채권구분으로 변경

In [11]:
# 개별 파일 pkl 만들기
# wd = r"D:\3.자산\전산 dataset"
# basedate = "230423"
# filename = "계좌조회새창_20230424_0918"
# x = pd.read_excel(join(wd, basedate, filename +".xlsx"), dtype={'채무자키':str, '계좌키':str, '타채무자키':str}).fillna("")
# x.to_pickle(join(wd, basedate, filename +".pkl"))

In [141]:
# 파일 읽기
# 채무자, 계좌, 보증인, 개회신복 기준데이터->차주와 보증인으로 나누 저장
wd = r"D:\3.자산\전산 dataset\대성"
################### 예수금 지울것(채무자,계좌)
basedate = "230430"
files = ("채무자조회새창_20230503_1536", "계좌조회정보", "보증인정보", "개인회생정보_진행중_계좌키분리_기준데이터", "신용회복전체리스트_대성_20230430")
# 개회의 겨우 계좌키 분리 파일로 읽을 것! 

# 파일 읽기
ext = ".xlsx"  
###################

# 개회-신복 충돌 정리(기준데이터 만들 때 각 담당자가 제거하기로 함)
# 개회제외 = {"차주" : ("20462830"), "보증인" : () } # 20462830 중복입금건. 신복이 기간이 더 남아서 신복으로 처리하기로 함
# 신복제외 = {"차주" : ("20451534", "20525880", "20525532"), "보증인" : ()} # "20451534", "20525880", "20525532" 신복 실효 확정 전에 개회 신청한 건

if ext == ".pkl" : # fillna()는 엑셀읽을 때 미리 해둬야함, 조정, 개회 진행중 시트만 읽어서 저장한 경우
    debt = pd.read_pickle(join(wd, basedate, files[0]+ext))
    account = pd.read_pickle(join(wd, basedate, files[1]+ext))
    grt = pd.read_pickle(join(wd, basedate, files[2]+ext))
    rehabilitation = pd.read_pickle(join(wd, basedate, files[3]+ext))
    credit = pd.read_pickle(join(wd, basedate, files[4]+ext))
    rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"]
    rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"]
    credit_d = credit[credit["채무구분"]=="주채무"]
    credit_grt = credit[credit["채무구분"]=="보증채무"]
else : 
    debt = pd.read_excel(join(wd, basedate, files[0]+ext), dtype={'채무자키':str, '타채무자키':str, '담당자키':str}).fillna("")
    account = pd.read_excel(join(wd, basedate, files[1]+ext), dtype={'채무자키':str, '계좌키':str, '타채무자키':str}).fillna("")
    grt = pd.read_excel(join(wd, basedate, files[2]+ext), dtype={'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}).fillna("")
    rehabilitation = pd.read_excel(join(wd, basedate, files[3]+ext), sheet_name="개인회생(진행)", dtype={'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}).fillna("")
    credit = pd.read_excel(join(wd, basedate, files[4]+ext), sheet_name="확정,미확정", dtype={'채무자키':str, '계좌키':str, '보증인키':str}).fillna("")
    rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"]
    rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"]
    credit_d = credit[credit["채무구분"]=="주채무"]
    credit_grt = credit[credit["채무구분"]=="보증채무"]
    
# 예수금 제외
debt.drop(debt[debt.성명=="예수금"].index, inplace=True)
debt.reset_index(drop=True, inplace=True)
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account.reset_index(drop=True, inplace=True)

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# pkl 저장
debt.to_pickle(join(wd, basedate, files[0]+".pkl"))
account.to_pickle(join(wd, basedate, files[1]+".pkl"))
grt.to_pickle(join(wd, basedate, files[2]+".pkl"))
rehabilitation.to_pickle(join(wd, basedate, files[3]+".pkl"))
credit.to_pickle(join(wd, basedate, files[4]+".pkl"))

In [142]:
# [check]개회신복 채무자키 중복 : 차주, 보증인별

# 차주
조정중복_차주 = pd.merge(rehabilitation_d[["채무자키","계좌키","인가/미인가","반영OPB", "접수일"]], credit_d[["채무자키","계좌키","진행구분","현재원금잔액","접수일자"]], on=["채무자키","계좌키"], how="inner")
display("차주", 조정중복_차주)

# 보증인
조정중복_보증인 = pd.merge(rehabilitation_grt[["채무자키","계좌키","인가/미인가","반영OPB", "접수일"]], credit_grt[["채무자키","계좌키","진행구분","현재원금잔액", "접수일자"]], on=["채무자키","계좌키"], how="inner")
display("보증인", 조정중복_보증인)

'차주'

,채무자키,계좌키,인가/미인가,반영OPB,접수일,진행구분,현재원금잔액,접수일자


'보증인'

,채무자키,계좌키,인가/미인가,반영OPB,접수일,진행구분,현재원금잔액,접수일자


In [165]:
# [check] 계좌종결건 조정채권 포함여부
종결된계좌키1 = account[account.종결일 != ""].계좌키
종결된계좌키2 = account[account.메모 != ""].계좌키
종결된계좌키 = pd.concat([종결된계좌키1, 종결된계좌키2])
check개회 = 종결된계좌키[종결된계좌키.isin(rehabilitation["계좌키"])]
check신복 = 종결된계좌키[종결된계좌키.isin(credit["계좌키"])]
check = pd.concat([check개회, check신복])
if len(check) > 0 :
    print(check)
    # 출력
    check.to_excel(join(wd, basedate, "9.종결vs조정.xlsx"), index=False)
    # 종결건 신복과 개회에서 제외하기
    rehabilitation = rehabilitation[~rehabilitation.계좌키.isin(check)]
    credit = credit[~credit.계좌키.isin(check)]
else : print("이상무")

이상무


In [ ]:
# 확정된 조정채권 저장하기
rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"]
rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"]
credit_d = credit[credit["채무구분"]=="주채무"]
credit_grt = credit[credit["채무구분"]=="보증채무"]

In [166]:
# 개회 >> 차주 : 계좌키, 보증인 : [계좌키 + 보증인이름]이 pk. 
# 신복 >> 보증인키 있으므로 이걸로. 누락계좌는 정상으로 나옴
# 개회 누락채권(일부계좌누락, 일부 채무자키(이런 경우가 있을까?)) 문제는 어떻게? : 일단 

# 보증인 상태변경
if "새보증인상태" in grt.columns : 
    grt["새보증인상태"] = ""
else : grt.insert(8,"새보증인상태","")

# '채무자키'열과 '보증인성명인'열을 기준으로 비교하여 값 변경
for i in tqdm(range(len(grt)), total=len(grt)):  
    # 종결건 보증인상태 유지
    
    # 개인회생
    temp = rehabilitation_grt[(rehabilitation_grt['채무자키'] == grt.at[i, '채무자키']) & (rehabilitation_grt['분납자성명인'] == grt.at[i, '보증인성명인'])]
    if len(temp) > 0:
        grt.at[i, '새보증인상태'] = "개인회생(확정)" if temp['인가/미인가'].values[0] == "인가" else "개인회생(진행중)"
    else : 
        # 신용회복
        temp = credit_grt[(credit_grt['보증인키'] == grt.at[i, '보증인키'])]
        if len(temp) > 0:
            grt.at[i, '새보증인상태'] = "신용회복("+ temp['상환방식'].values[0]+")" if temp['진행구분'].values[0] == "확정" else "신용회복(진행중)"
        else :
            # 무담보
            grt.at[i, '새보증인상태'] = grt.at[i, '보증인상태']

# move = grt.pop('새보증인상태')
# grt.insert(8,'새보증인상태', move)

100%|██████████| 415/415 [00:00<00:00, 2129.78it/s]


In [6]:
# 저장 : 보증인 수정할 것 엑셀 저장
grt[grt.새보증인상태 != grt.보증인상태].loc[:,["채무자키", "계좌키","보증인키","매각사구분","보증인성명인","주민번호인","계좌번호","보증구분","새보증인상태","보증인상태","보증금액","보증잔액","보증한도액"]].to_excel(join(wd, basedate, "9.보증인상태변경할것.xlsx"), index=False)
# # 새보증인상태 추가한 파일 저장
# grt.to_pickle(join(wd, basedate, files[2]+"_새보증인상태.pkl"))
grt.to_excel(join(wd, basedate, files[2]+"_새보증인상태.xlsx"), index=False)

In [169]:
# 계좌에 새채무상태 만들기

# 계좌조회새창의 채무상태는 전산의 채무자상태값을 불러온 것임.(즉 채무자상태이지 채권상태가 아님)
# 1. 새채무상태 채우키
# if 종결일에 값이 있으면 종료
# else if 조정기준데이터에서 불러오고 
#      else ~~~
#   
# 2. 수정목록 작성하기

if "새채무상태" not in account.columns : account.insert(8,"새채무상태", "")
종결처리안된종료건 = []

for i in tqdm(range(len(account)), total = len(account)):
    account_key = account.at[i, '계좌키']
    # 종결처리 안 된 계좌
    if account.at[i, '종결일'] == "" :
        # 종결처리 안 됐지만 종료인 계좌(메모 "종료")
        if account.at[i, "메모"] != "" :
            account.at[i, '새채무상태'] = account.at[i, "메모"]
            종결처리안된종료건.append(account.at[i, '계좌키'])
        # 미종결 or 보증인까지 고려해야 하는 것
        else : 
            # 개인회생
            temp = rehabilitation_d[(rehabilitation_d['계좌키'] == account_key)]
            if  len(temp) > 0:
                account.at[i, '새채무상태'] = "개인회생(확정)" if temp['인가/미인가'].values[0] == "인가" else "개인회생(진행중)"
            else : 
                # 신용회복
                temp = credit_d[(credit_d['계좌키'] == account.at[i, '계좌키'])]
                if len(temp) > 0:
                    account.at[i, '새채무상태'] = "신용회복("+ temp['상환방식'].values[0]+")" if temp['진행구분'].values[0] == "확정" else "신용회복(진행중)"
                else :
                    # 일단 빈칸으로 두고 뒤에서 수정 # 무담보 또는 종료(면책)
                    account.at[i, '새채무상태'] = ""
    # 종결처리 된 계좌
    else :
        account.at[i, '새채무상태'] = "종료"

# move = account.pop('새보증인상태')
# account.insert(8,'새보증인상태', move)

100%|██████████| 6456/6456 [00:02<00:00, 2849.69it/s]


In [171]:
# [ckeck]종결처리해야하는 계좌 출력
if len(종결처리안된종료건)>0 :
    pd.DataFrame(종결처리안된종료건).to_excel(join(wd, basedate, "9.종결처리안된종료건.xlsx"), index=False)

### 전체 원데이터 만들기(종합)

In [209]:
raw_data = pd.DataFrame(None)
raw_data["채무자키"] = account.채무자키
raw_data["계좌키"] = account.계좌키
raw_data["타채무자키"] = account.타채무자키

raw_data["채권구분"] = account.채권구분
#raw_data["매각사"] = account.매각사     # 매각사 구분 있다.
raw_data["새채무상태"] = account.새채무상태
raw_data["채무상태"] = account.채무상태
raw_data["담당자"] = account.담당자
raw_data["성명"] = account.채무자명

raw_data["주민등록번호"] = account.채무자주민번호 #.apply(lambda x : x[:8])
raw_data["계좌번호"] = account.계좌번호
raw_data["원채권사"] = account.원채권사
raw_data["매각사구분"] = account.매각사구분
raw_data["계정과목"] = account.계정과목
raw_data["최초원금"] = account.최초원금
raw_data["최초법비용"] = account.최초법비용
raw_data["최초미수이자"] = account.최초미수이자
raw_data["최초연체이자율"] = account.최초연체이자율
raw_data["현재원금"] = account.현재원금
raw_data["현재법비용"] = account.현재법비용
raw_data["OPB"] = account[["현재원금","현재법비용"]].sum(axis=1)
raw_data["현재미수이자"] = account.현재미수이자
raw_data["현재연체이자"] = account.현재연체이자
raw_data["현재합계"] = account.현재합계
raw_data["회수합계"] = account.회수합계
raw_data["최초대출일"] = account.최초대출일
raw_data["최초연체일"] = account.최초연체일 # 없는 경우도 있음
raw_data["매입일"] = account.매입일
raw_data["시효완성일"] = account.시효완성일

# 채무자조회에서 불러올 항목
debt_temp = debt[["채무자키", "개인법인"]]#, "연령", "자택우편번호", "자택주소"]]
raw_data = pd.merge(raw_data, debt_temp, on="채무자키", how="left")

# grt에서 각 계좌별 보증인상태 불러오기
names = grt.groupby('계좌키')['보증인성명인'].apply(lambda x: ','.join(x)) # 고유값만 남기려면 join의 매개변수로 x.unique()를 사용
status = grt.groupby('계좌키')['보증인상태'].apply(lambda x: ','.join(x))

raw_data = raw_data.merge(names, how='left', on='계좌키')
raw_data = raw_data.merge(status, how='left', on='계좌키')

raw_data["보증인성명인"] = raw_data["보증인성명인"].fillna('')
raw_data["보증인상태"] = raw_data["보증인상태"].fillna('')

In [210]:
# 새채무상태 마무리
print("전",raw_data.새채무상태.value_counts())
채무상태종료 = re.compile("종결|시효완성|신용회복\(완제\)|환매|매각")
담당자종료 = re.compile("종결|상각|포기채권|매각|환매|파산|회계파산")
채무상태면책 = re.compile("개인회생\(면책\)|파산\(면책\)")
보증인종료 = re.compile("파산\(면책\)|종결|시효완성|개인회생\(면책\)|추심제외|시효|보증면탈|사망\(상속포기\)")

for i in tqdm(range(len(raw_data)), total = len(raw_data)):
    if raw_data.at[i, "새채무상태"] == "" :
        # 종료
        if 채무상태종료.search(raw_data.at[i, "채무상태"]) or 담당자종료.search(raw_data.at[i, "담당자"]) :
            raw_data.at[i, "새채무상태"] = "종료"
        # 차주면책
        elif 채무상태면책.search(raw_data.at[i, "채무상태"]) :
            # 보증인 없으면
            if raw_data.at[i, "보증인상태"] == "" :
                raw_data.at[i, "새채무상태"] = "종료"
            else : # 보증인종료 이외의 건이 있는지
                temp = 보증인종료.sub("",raw_data.at[i, "보증인상태"])
                if len(re.sub(",", "", temp)) == 0 : # 보증인도 모두 종료/면책
                    raw_data.at[i, "새채무상태"] = "종료"
                else :
                    raw_data.at[i, "새채무상태"] = re.sub("사업자", "", raw_data.at[i, "개인법인"])
        else :
            raw_data.at[i, "새채무상태"] = re.sub("사업자", "", raw_data.at[i, "개인법인"])
print("후",raw_data.새채무상태.value_counts())

전              3727
종료           1010
신용회복(개인)      949
개인회생(확정)      639
개인회생(진행중)      57
신용회복(프리)       50
신용회복(진행중)      24
Name: 새채무상태, dtype: int64


100%|██████████| 6456/6456 [00:00<00:00, 25806.21it/s]

후 종료           2420
개인           2207
신용회복(개인)      949
개인회생(확정)      639
법인            110
개인회생(진행중)      57
신용회복(프리)       50
신용회복(진행중)      24
Name: 새채무상태, dtype: int64


In [ ]:
# [ckeck]계좌별 채무상태 수정할 것 엑셀 저장
# 범주가 다르게 적용되어서 현재는 의미가 없다. 보완이 필요
# raw_data[raw_data.새채무상태 != raw_data.채무상태].loc[:,["채무자키", "계좌키","매각사구분","새채무상태","채무상태"]].to_excel(join(wd, basedate, "9.계좌별채무상태달라진것.xlsx"), index=False)

In [211]:
# 최종OPB 작성하기

개회OPB = rehabilitation_d.loc[:, ["계좌키", "반영OPB"]].rename(columns={"반영OPB" : "최종OPB"})
신복OPB = credit_d.loc[:, ["계좌키", "현재원금잔액"]].rename(columns={"현재원금잔액" : "최종OPB"})
# 개회OPB와 신복OPB 병합
merged_opb = 개회OPB.merge(신복OPB, on=['계좌키','최종OPB'], how='outer')
# raw_data와 병합
raw_data = raw_data.merge(merged_opb, on='계좌키', how='outer')

# 조정이 아닌 경우, 종료이면 0, 아니면 OPB
new_col = np.where(raw_data['새채무상태'] == '종료', 0, raw_data['OPB'])
raw_data.loc[raw_data['최종OPB'].isnull(), '최종OPB'] = new_col[raw_data['최종OPB'].isnull()]

In [212]:
# 전체 저장하기
raw_data.to_excel(join(wd, basedate, "2.대성_전체_원데이터_"+basedate+".xlsx"), index=False)
# 특정 매각사만 저장하기
# cond = "스마트저축-01" ###################################
# raw_data[raw_data.매각사구분==cond].to_excel(join(wd, cond+"_원데이터_"+basedate+".xlsx"), index=False)

In [186]:
# [check] 더블 채무상태
채무자별채무상태 = raw_data.groupby('주민등록번호')['새채무상태'].apply(lambda x:",".join(x.unique()))
check_채무상태 = 채무자별채무상태[채무자별채무상태.str.contains(',')]
# 출력
if len(check_채무상태) > 0 : check_채무상태.to_excel(join(wd, basedate, "9.더블채무상태.xlsx"))

In [ ]:
# 제출용 시트 작성하기-개인회생 
re_d = rehabilitation_d.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가/미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","비고"]]
re_grt = rehabilitation_grt.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가/미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","비고"]]
# re_d.insert(0, "순번", range(1,len(re_d)+1))
# re_grt.insert(0, "순번", range(1,len(re_grt)+1))
# 저장
with pd.ExcelWriter(join(wd, basedate, "10.제출용 개인회생.xlsx"), engine='openpyxl', mode = 'w') as writer :
    re_d.to_excel(writer, sheet_name="차주", index=False)
    re_grt.to_excel(writer, sheet_name="보증인", index=False)

In [ ]:
# 제출용 시트 작성하기-신용회복
# 기준데이터에서 삭제한 것이 있을 때 유의미 & 차주 보증인 구분

# 날짜열 문자열로
date_cols = credit_d.select_dtypes(include='datetime').columns
credit_d.loc[:,date_cols] = credit_d[date_cols].astype(str)
date_cols = credit_grt.select_dtypes(include='datetime').columns
credit_grt.loc[:,date_cols] = credit_grt[date_cols].astype(str)

# 저장
with pd.ExcelWriter(join(wd, basedate, "10.제출용 신용회복.xlsx"), engine='openpyxl', mode = 'w') as writer :
    credit_d.to_excel(writer, sheet_name="차주", index=False)
    credit_grt.to_excel(writer, sheet_name="보증인", index=False)

In [216]:
# 요약 
pd.options.display.float_format = '{:.0f}'.format
# 특정 POOL
# pool계좌키 = pd.read_excel("c://users/sl/Desktop/계좌키.xlsx", dtype={"계좌키":str})
# pool = pd.merge(raw_data,pool계좌키, on='계좌키', how='inner')
# result = pool.groupby('새채무상태').agg({'계좌키':'count', '최종OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','종료'])
# result
# 전체
raw_data.groupby('새채무상태').agg({'계좌키':'count', '최종OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','종료'])

,계좌키,최종OPB
새채무상태,,
개인,2207,14292580546
법인,110,2515212018
개인회생(확정),639,536201755
개인회생(진행중),57,480869119
신용회복(개인),949,3332121693
신용회복(프리),50,300512710
신용회복(진행중),24,81070088
종료,2420,0


In [ ]:
############## 끝 ###################

In [176]:
path = r"D:\3.자산\전산 dataset\대성\POOL1_OK저축"
file = "1. OK저축은행_전체계좌_230316_완.xlsx"
raw = pd.read_excel(join(path,file), sheet_name="Sheet1", dtype={'채무자키':str, '계좌키':str}).fillna('')

In [177]:
raw

,채무자키,계좌키,구분,채무자키중복,계좌키중복
0,10622620,1568579,무담보,1,1
1,10622627,1570572,무담보,1,1
2,10622640,1572061,무담보,1,1
3,10622644,1572198,무담보,1,1
4,10622646,1568537,무담보,1,1
...,...,...,...,...,...
1638,10625674,1572044,종결,1,1
1639,10623511,1572045,종결,1,1
1640,10622856,1572175,종결,1,1
1641,10625757,1572231,종결,1,1


In [179]:
pool = raw.groupby('채무자키')['구분'].apply(lambda x: ','.join(x.unique())) # 고유값만 남기려면 join의 매개변수로 x.unique()를 사용
pool.to_excel(join(path, "xxx.xlsx"))

In [12]:
len(raw)

4445

In [13]:
raw.to_excel(join(path, "채무자별.xlsx"), index=False)

# 이전 파일

In [2]:
path_dir_excel = r"D:\3.자산\전산 dataset\230409(일)"
filename_excel = r"2.솔림헬프_전체_원데이터_230411.xlsx"
df = pd.read_excel(join(path_dir_excel, filename_excel)).fillna("")
df["계좌키"] = df["계좌키"].apply(str)
df["채무자키"] = df["채무자키"].apply(str)
total = len(df.index)
print(total)

78158


In [ ]:
# 관리여부 : 종료, 종료(면책), 정상(넓은범위) 작성중..
# 관리여부 = []
# for i, v in df.iterrows() :
#     if re.search("종결|매각|환매", v.채무상태) :
#         관리여부.append("종료")
#     elif re.search("면책", v.채무상태) :
#         if v.보증인상태 == "" :
#             관리여부.append("종료(면책)")
            

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78157 entries, 0 to 78156
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   계좌키     78157 non-null  object
 1   채무자키    78157 non-null  object
 2   채권구분    78157 non-null  object
 3   일부조정    78157 non-null  object
 4   차주신복여부  78157 non-null  bool  
 5   차주개회여부  78157 non-null  bool  
 6   정상/종료   78157 non-null  object
 7   채무상태    78157 non-null  object
 8   보증인상태   78157 non-null  object
dtypes: bool(2), object(7)
memory usage: 4.3+ MB


In [52]:
my_list = ['']
new_list = list(filter(lambda x: x != "", my_list))
if len(new_list) == 0:
    print("요소의 개수가 0인 리스트입니다.")
else:
    print(new_list)

요소의 개수가 0인 리스트입니다.


### 전체원데이터

In [2]:
# 채무자키 20409775 2301 > 2031년으로 수정해야함 안그럼 에러 아래 날짜변경에서 에러남##############
# 엑셀을 pickle파일로
wd = r'D:\3.자산\전산 dataset\230423\temp'
for f in os.listdir(wd) :
    if os.path.isfile(join(wd,f)) :
        if re.search(".xlsx", f) :
            df = pd.read_excel(join(wd, f)).fillna("")
            df.to_pickle(join(wd, os.path.splitext(f)[0]+".pkl"))

In [3]:
# 파일 명칭 자동 저장
account_f, debt_f, grt_f, law_f, collection_f = "", "", "", "", ""
for root, dirs, files in os.walk(wd) :
    for f in files :
        if re.search(".pkl", f) :
            if re.search("계좌",f) : account_f = f
            elif re.search("채무자",f) : debt_f = f
            elif re.search("보증인",f) : grt_f = f
            elif re.search("법조치",f) : law_f = f
            elif re.search("입금",f) : collection_f = f

In [4]:
account = pd.read_pickle(join(wd, account_f))
debt = pd.read_pickle(join(wd, debt_f))
#collection = pd.read_pickle(join(wd,collection_f))
############################
# 매각사 = "전북은행"
# account = account[account.매각사.str.contains(매각사)]
# debt = debt[debt.매각사.str.contains(매각사)]
############################
print(len(account), len(debt))

78157 70795


In [5]:
# 채무자키 20409775 2301 > 2031년으로 수정해야함 안그럼 에러
# account.시효완성일 = pd.to_datetime(account.시효완성일)
# collection.입금일 = pd.to_datetime(collection.입금일)

In [5]:
raw_data = pd.DataFrame(None)
raw_data["채무자키"] = account.채무자키.apply(str)
raw_data["계좌키"] = account.계좌키.apply(str)
raw_data["타채무자키"] = account.타채무자키.apply(str)

raw_data["채권구분"] = account.채권구분
#raw_data["매각사"] = account.매각사
raw_data["채무상태"] = account.채무상태
raw_data["담당자"] = account.담당자
raw_data["성명"] = account.채무자명

raw_data["주민등록번호"] = account.채무자주민번호.apply(lambda x : x[:8])
raw_data["계좌번호"] = account.계좌번호
raw_data["원채권사"] = account.원채권사
raw_data["매각사구분"] = account.매각사구분
raw_data["계정과목"] = account.계정과목
raw_data["최초원금"] = account.최초원금
raw_data["최초법비용"] = account.최초법비용
raw_data["최초미수이자"] = account.최초미수이자
raw_data["최초연체이자율"] = account.최초연체이자율
raw_data["현재원금"] = account.현재원금
raw_data["현재법비용"] = account.현재법비용
raw_data["OPB"] = account[["현재원금","현재법비용"]].sum(axis=1)
raw_data["현재미수이자"] = account.현재미수이자
raw_data["현재연체이자"] = account.현재연체이자
raw_data["현재합계"] = account.현재합계
raw_data["회수합계"] = account.회수합계
raw_data["최초대출일"] = account.최초대출일
raw_data["최초연체일"] = account.최초연체일 # 없는 경우도 있음
raw_data["매입일"] = account.매입일
raw_data["시효완성일"] = account.시효완성일

In [6]:
debt.채무자키 = debt.채무자키.apply(str)
debt = debt[["채무자키", "개인법인", "연령", "자택우편번호", "자택주소","보증인성명", "보증인상태"]]
raw_data = pd.merge(raw_data, debt, on="채무자키", how="inner")
raw_data.개인법인.isnull().value_counts()

False    78157
Name: 개인법인, dtype: int64

In [7]:
# 전체 저장하기
raw_data.to_excel(join(wd, "2.솔림헬프_전체_원데이터_"+datetime.today().strftime("%y%m%d")+".xlsx"), index=False)

In [15]:
# 특정 매각사만 저장하기
cond = "스마트저축-01" ###################################
raw_data[raw_data.매각사구분==cond].to_excel(join(wd, cond+"_"+datetime.today().strftime("%Y%m%d")+".xlsx"), index=False)

### 전체 원데이터에서 채무자키, 계좌키 추출하기

In [9]:
# 파일 읽기
df_all = raw_data
# # 또는
# df_all = pd.read_excel(join(wd, "2.2.솔림헬프_전체_원데이터_20230223.xlsx"))

In [21]:
##################
keyListFile = "POO8채무자키리스트" # 채무자키 또는 계좌키 칼럼명과 그 리스트만 있는 엑셀파일 ex) pool1
keyName = "채무자키"
##################
keys = pd.read_excel(join(wd, keyListFile+".xlsx"))["채무자키"].astype(str)

In [23]:
result = pd.merge(keys, df_all, how="left")
result.to_excel(join(wd, "2."+keyListFile+"_원데이터_"+datetime.today().strftime("%y%m%d")+".xlsx"), index=False)

### 차입자산 POOL별 중복확인

In [102]:
dfdf = pd.read_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산\참고\P5 DataDisk 수정후,by채무자키_230303","1.차입자산 리스트.xlsx"))

In [109]:
중복 = dfdf[dfdf.duplicated(['채무자키'],keep=False)].sort_values(['채무자키','POOL_NO']) # 이렇게 해야 1-8, 8-1 이 안나오고 1-8로만
#.to_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산","중복체크.xlsx"), index=False)

In [110]:
#중복["계좌키"] = 중복["계좌키"].astype(int).astype(str)
중복["채무자키"] = 중복["채무자키"].astype(int).astype(str)
중복["반복횟수"] = 중복.groupby(['채무자키'])['POOL_NO'].transform('count')

In [111]:
중복

,채무자키,POOL_NO,최종본,반복횟수
62644,20407795,POOL_05,NaN,2
62643,20407795,POOL_08,X,2
62626,20407813,POOL_01,X,2
62625,20407813,POOL_05,NaN,2
62602,20407840,POOL_01,X,2
...,...,...,...,...
12108,20466470,POOL_14,NaN,2
11900,20469120,POOL_02,NaN,2
11899,20469120,POOL_06,NaN,2
11694,20469325,POOL_02,NaN,2


In [112]:
중복 = 중복.reset_index(drop=True, inplace=False)
중복

,채무자키,POOL_NO,최종본,반복횟수
0,20407795,POOL_05,NaN,2
1,20407795,POOL_08,X,2
2,20407813,POOL_01,X,2
3,20407813,POOL_05,NaN,2
4,20407840,POOL_01,X,2
...,...,...,...,...
875,20466470,POOL_14,NaN,2
876,20469120,POOL_02,NaN,2
877,20469120,POOL_06,NaN,2
878,20469325,POOL_02,NaN,2


In [66]:
중복.to_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산","중복체크.xlsx"), index=False)

In [2]:
중복 = pd.read_excel(r'D:\3.자산\POOL별 관리자산\무차입자산\중복체크.xlsx')
중복

,계좌키,채무자키,POOL_
0,200919485,20407790,POOL_05
1,200919485,20407790,POOL_05
2,200919497,20407882,POOL_05
3,200919497,20407882,POOL_08
4,200919997,20407883,POOL_01
...,...,...,...
1725,201046260,20515407,POOL_13
1726,201046280,20515427,POOL_13
1727,201046280,20515427,POOL_13
1728,201046336,20515483,POOL_13


In [113]:
채무자키, 반복횟수, 반복POOL = [], [], []
이전채무자키 = ""
for i, v in 중복.iterrows() :
    if v.채무자키 != 이전채무자키 :
        이전채무자키 = v.채무자키
        채무자키.append(v.채무자키)
        반복횟수.append(v.반복횟수)
        반복POOL.append(list(중복.groupby('채무자키').get_group(v.채무자키).POOL_NO))

result = pd.DataFrame({'채무자키':채무자키, '반복횟수':반복횟수, '반복POOL':반복POOL})
result.to_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산","중복체크_1.xlsx"), index=False)